**Orchestrate a multi-agent system:--** a multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web!

In [ ]:
!pip install smolagents[toolkit] --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

## Create a Web search tool

In [ ]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool

@tool
def visit_webpage(url: str) -> str:
  """Visits a webpage at the given URL and returns its content as markdown string.

  Args:
      url: The URL of the webpage to visit.

  Returns:
      The content of the webpage converted to Markdown, or an error message if the request fails.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()

    markdown_content = markdownify(response.text).strip()

    markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

    return markdown_content

  except RequestException as e:
    return f"Error fetching the webpage: {str(e)}"
  except Exception as e:
    return f"An unexpected error occurred: {str(e)}"

In [ ]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[500:])

ct us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")

Contribute

* [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")
* [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")
* [Community portal](/wiki/Wikipedia:Community_portal "The hub for editors")
* [Recent changes](/wiki/Special:RecentChanges "A list of recent changes to Wikipedia [r]")
* [Upload file](/wiki/Wikipedia:File_upload_wizard "Add images or other media for use on Wikipedia")
* [Special pages](/wiki/Special:SpecialPages)

[![](/static/images/icons/wikipedia.png)

![Wikipedia](/static/images/mobile/copyright/wikipedia-wordmark-en.svg)
![The Free Encyclopedia](/static/images/mobile/copyright/wikipedia-tagline-en.svg)](/wiki/Main_Page)

[Search](/wiki/Special:Search "Search Wikipedia [f]")

Search

Appearance

* [Donate](https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en)
* [Create account](/w/index.php?

## Build our multi-agent system

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
)

model = InferenceClientModel(model_id=model_id)

web_agent = ToolCallingAgent(
    tools = [WebSearchTool(), visit_webpage],
    model = model,
    max_steps = 10,
    name = "web_search_agent",
    description= "Runs web searches for you.",
)

In [ ]:
manager_agent = CodeAgent(
    tools = [],
    model = model,
    managed_agents = [web_agent],
    additional_authorized_imports=['time', 'numpy', 'pandas'],   # just in case the agent needs these packages.
)

In [ ]:
# Run the system
answer = manager_agent.run("If LLM training continues to scale up at the current rhythm until 2030, what would be the electric power in GW required to power the biggest training runs by 2030? What would that correspond to, compared to some countries> Please provide a source for any numbers used.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If LLM training continues to scale up at the current rhythm until 2030, what would be the electric power in GW  │
│ required to power the biggest training runs by 2030? What would that correspond to, compared to some countries> │
│ Please provide a source for any numbers used.                                                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  llm_power_current = web_search_agent("current power consumption of large-scale LLM training runs")               
  print(llm_power_current)                                                                                         
                                                                                                                   
  llm_power_projected = web_search_agent("projected power consumption of LLM training runs in 2030")               
  print(llm_power_projected)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ current power consumption of large-scale LLM training runs                                                      │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 2: Duration 25.05 seconds| Input tokens: 6,347 | Output tokens: 1,074]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-682811c7-3e1e7d6e13d8ca8263966844;b9d9d63b-f8f0-4a8b-ab20-b6a9913ff75b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 3: Duration 0.07 seconds| Input tokens: 9,959 | Output tokens: 1,614]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-682811c7-3e1e7d6e13d8ca8263966844;b9d9d63b-f8f0-4a8b-ab20-b6a9913ff75b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.